# Economic Dispatch (ED) quick run on Binder

This notebook mirrors the ED example from your docs and is meant to be launched via Binder.


In [ ]:
using Pkg
println("Julia version: ", VERSION)
# If you later add Manifest.toml, this will use exact versions.
Pkg.activate("..")
Pkg.instantiate()
using JuMP, Ipopt, DataFrames, IJulia

# --- ED example ---
D = 100
genset = 1:3
C = [10, 20, 30]
G_min = [0, 0, 0]
G_max = [50, 50, 50]

m = Model(Ipopt.Optimizer)
@variable(m, g[i in genset])
@objective(m, Min, sum(C[i] * g[i] for i in genset))
@constraint(m, SDbalance, sum(g[i] for i in genset) == D)
@constraint(m, glb[i in genset], g[i] >= G_min[i])
@constraint(m, gub[i in genset], g[i] <= G_max[i])
optimize!(m)

results = DataFrame(
    Generator = genset,
    Cost = C,
    Dispatch = round.(value.(g).data, digits=2),
    Gmax = G_max,
)
println(results)
println("Total cost = ", objective_value(m))
println("SMP (dual of balance) = ", dual.(SDbalance))
